In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from sklearn.model_selection import StratifiedKFold, ParameterGrid
sys.path.append(os.environ['CMS_ROOT'])
from cms_modules.utils import model_summary_to_string, args_to_dict
from cms_modules.logging import Logger

import tensorflow as tf
EarlyStopping = tf.keras.callbacks.EarlyStopping
TensorBoard = tf.keras.callbacks.TensorBoard
# ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau

ecbdl14_root = '/Users/jujohnson/git/ECBDL14-Classification'
# ecbdl14_root = '/home/jjohn273/git/ECBDL14-Classification/'
sys.path.append(ecbdl14_root)
from model import create_model
from CustomCallbacks import KerasRocAucCallback

/var/web_services/galaxy/jupyter_conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
E0123 21:38:52.537112 46912496279168 ultratb.py:147] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/var/web_services/galaxy/jupyter_conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-b98c6d86d49d>", line 11, in <module>
    import tensorflow as tf
  File "/home/jjohn273/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 101, in <module>
    from tensorflow_core import *
  File "/home/jjohn273/.local/lib/python3.6/site-packages/tensorflow_core/__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 947, in _find_and_load_unlocked
  File "/home/jjohn273/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/home/jjohn273/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 44, in

TypeError: __new__() got an unexpected keyword argument 'serialized_options'

### Define Model Config

In [ ]:
config = {
    'hidden_layers': [128, 128, 64, 32],
    'learn_rate': 0.001,
    'batch_size': 256,
    'dropout_rate': 0.2,
    'batchnorm': True
}

epochs = 40
debug = True

### Define I/O Paths

In [ ]:
# inputs
data_path = os.path.join(ecbdl14_root, 'data/ecbdl14.onehot.sample.hdf')
data_key = 'train'
# outputs
now = datetime.datetime.today()
ts = now.strftime("%m%d%y-%H%M%S")
architecture_output = 'model-architecture.json'

### Init Logger

In [ ]:
log_file = f'logs/{ts}.txt'
logger = Logger(log_file)
logger.log_time('Starting trainer')
logger.write_to_file()

### Load Data

In [2]:
df = pd.read_hdf(data_path, data_key)
logger.log_time(f'Loaded data with shape {df.shape}').write_to_file()
if debug:
    y, x = df[:10000]['target'], df[:10000].drop(columns=['target'])
else:
    y, x = df['target'], df.drop(columns=['target'])

E0123 21:38:56.208641 46912496279168 ultratb.py:147] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/var/web_services/galaxy/jupyter_conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-9f2144df28e9>", line 1, in <module>
    df = pd.read_hdf(data_path, data_key)
NameError: name 'data_path' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/web_services/galaxy/jupyter_conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/web_services/galaxy/jupyter_conda/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of

NameError: name 'data_path' is not defined

In [ ]:
def write_model(model, path):
    json = model.to_json()
    with open(path, 'w') as out:
        out.write(json)

### Iterate Over Runs

In [ ]:
for run in runs:
    logger.log_time(f'Starting run {run}')).write_to_file()
    # prepare input data
    x_train, y_train = x.iloc[train_index].values, y.iloc[train_index].values
    x_valid, y_valid = x.iloc[validate_index].values, y.iloc[validate_index].values
    input_dim = x_train.shape[1]
        
    # create model and save architecture to file if does not exist
    dnn = create_model(input_dim, config)
    if not os.path.isfile(architecture_output):
        write_model(dnn, architecture_output)
        print(f'Model architecture saved to {architecture_output}')

    # train model
    logger.log_time('Starting training...').write_to_file()
    dnn.fit(x_train, y_train, epochs=epochs, verbose=0)
    logger.log_time('Trainin complete!').write_to_file()
    
    # save weights
    out = f'{experiment}-{run}-model.h5'
    dnn.save_weights(out)
    logger.log_time(f'Model weights saved to {out}')

In [ ]:
logger.log_time('Job complete...').write_to_file()